In [6]:
import pandas as pd
from watttime import WattTimeMyAccess
import os

In [1]:
import requests
import json
import pandas as pd

headers = {'auth-token': "vCC6qF584P3ua2blVREx"}

resp = requests.get("https://api.electricitymap.org/v3/zones")
zones = resp.json()

/Users/jacobgoldverg/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:

df = pd.DataFrame(zones)
# print(df)
print(df['US'])

zoneName       United States
countryName              NaN
displayName              NaN
Name: US, dtype: object
zoneName       Canada
countryName       NaN
displayName       NaN
Name: CA, dtype: object


In [5]:
us_zones = {
    zone_code: zone_data
    for zone_code, zone_data in zones.items()
    if zone_code.startswith('US')
    or zone_code == 'US'
    or zone_data.get('countryName') in ('USA', 'United States')
    or zone_code.startswith('CA')
    or zone_code == 'CA'
    or zone_data.get('countryName') in ('CA', 'Canada')
}
print(len(us_zones))
print(us_zones.keys())

75
dict_keys(['CA', 'CA-AB', 'CA-BC', 'CA-MB', 'CA-NB', 'CA-NL', 'CA-NS', 'CA-NT', 'CA-NU', 'CA-ON', 'CA-PE', 'CA-QC', 'CA-SK', 'CA-YT', 'US', 'US-AK', 'US-AK-SEAPA', 'US-CAL-BANC', 'US-CAL-CISO', 'US-CAL-IID', 'US-CAL-LDWP', 'US-CAL-TIDC', 'US-CAR-CPLE', 'US-CAR-CPLW', 'US-CAR-DUK', 'US-CAR-SC', 'US-CAR-SCEG', 'US-CAR-YAD', 'US-CENT-SPA', 'US-CENT-SWPP', 'US-FLA-FMPP', 'US-FLA-FPC', 'US-FLA-FPL', 'US-FLA-GVL', 'US-FLA-HST', 'US-FLA-JEA', 'US-FLA-SEC', 'US-FLA-TAL', 'US-FLA-TEC', 'US-HI', 'US-MIDA-PJM', 'US-MIDW-AECI', 'US-MIDW-LGEE', 'US-MIDW-MISO', 'US-NE-ISNE', 'US-NW-AVA', 'US-NW-BPAT', 'US-NW-CHPD', 'US-NW-DOPD', 'US-NW-GCPD', 'US-NW-GRID', 'US-NW-IPCO', 'US-NW-NEVP', 'US-NW-NWMT', 'US-NW-PACE', 'US-NW-PACW', 'US-NW-PGE', 'US-NW-PSCO', 'US-NW-PSEI', 'US-NW-SCL', 'US-NW-TPWR', 'US-NW-WACM', 'US-NW-WAUW', 'US-NY-NYIS', 'US-SE-SEPA', 'US-SE-SOCO', 'US-SW-AZPS', 'US-SW-EPE', 'US-SW-PNM', 'US-SW-SRP', 'US-SW-TEPC', 'US-SW-WALC', 'US-TEN-TVA', 'US-TEX-ERCO', 'VI'])


In [6]:
from io import StringIO

#https://data.electricitymaps.com/2025-04-03/US-NW-AVA_2024_hourly.csv
base_url = "https://data.electricitymaps.com/2025-04-03/{zone}_2024_hourly.csv"
us_zones_data = {}
for zone_key in us_zones.keys():  # Assuming us_zones is your list of zone codes
    try:
        # Format the URL with current date and zone
        formatted_url = base_url.format(
            zone=zone_key
        )
        print(formatted_url)
        # Make the request
        resp = requests.get(formatted_url, headers=headers)
        resp.raise_for_status()  # Raise exception for bad status codes

        # Parse CSV content into DataFrame
        df = pd.read_csv(StringIO(resp.text))
        # Store the DataFrame
        us_zones_data[zone_key] = df

        print(f"Successfully downloaded data for {zone_key}")

    except Exception as e:
        print(f"Failed to download {zone_key}: {str(e)}")
        us_zones_data[zone_key] = None  # Store None for failed downloads


print(us_zones_data)
df = pd.concat(us_zones_data.values(), ignore_index=True)
df = df.rename(columns={'Datetime (UTC)': 'datetime', 'Zone name': 'zone_name', 'Zone id': 'zone_id', 'Carbon intensity gCO₂eq/kWh (direct)': 'ci', 'Carbon intensity gCO₂eq/kWh (Life cycle)': 'ci_lifecycle'})
df.to_csv('data/historical_data/2024_ZONES_DF.csv', index=False)
print(len(df))
print(df)

https://data.electricitymaps.com/2025-04-03/CA_2024_hourly.csv
Successfully downloaded data for CA
https://data.electricitymaps.com/2025-04-03/CA-AB_2024_hourly.csv
Successfully downloaded data for CA-AB
https://data.electricitymaps.com/2025-04-03/CA-BC_2024_hourly.csv
Successfully downloaded data for CA-BC
https://data.electricitymaps.com/2025-04-03/CA-MB_2024_hourly.csv
Successfully downloaded data for CA-MB
https://data.electricitymaps.com/2025-04-03/CA-NB_2024_hourly.csv
Successfully downloaded data for CA-NB
https://data.electricitymaps.com/2025-04-03/CA-NL_2024_hourly.csv
Successfully downloaded data for CA-NL
https://data.electricitymaps.com/2025-04-03/CA-NS_2024_hourly.csv
Successfully downloaded data for CA-NS
https://data.electricitymaps.com/2025-04-03/CA-NT_2024_hourly.csv
Successfully downloaded data for CA-NT
https://data.electricitymaps.com/2025-04-03/CA-NU_2024_hourly.csv
Successfully downloaded data for CA-NU
https://data.electricitymaps.com/2025-04-03/CA-ON_2024_hourly

In [7]:
import geopandas as gpd

world_em_df = gpd.read_file('config/geojson/world.geojson')
print(world_em_df['zoneName'].unique())

['SE-SE4' 'SE-SE3' 'SE-SE2' 'SE-SE1' 'XX' 'AE' 'AF' 'AL' 'AD' 'AO' 'AG'
 'AR' 'AM' 'AT' 'AU-NSW' 'AU-NT' 'AU-QLD' 'AU-SA' 'AU-TAS' 'AU-TAS-KI'
 'AU-TAS-FI' 'AU-TAS-CBI' 'AU-VIC' 'AU-WA' 'AX' 'AZ' 'BA' 'BB' 'BD' 'BE'
 'BH' 'BJ' 'BO' 'BR-CS' 'BR-N' 'BR-NE' 'BR-S' 'BS' 'BT' 'BW' 'BY' 'BZ'
 'BN' 'BG' 'BF' 'BI' 'CA-AB' 'CA-BC' 'CA-MB' 'CA-NB' 'CA-NL' 'CA-NS'
 'CA-ON' 'CA-PE' 'CA-QC' 'CA-SK' 'CA-NT' 'CA-NU' 'CA-YT' 'CF' 'CG' 'CH'
 'CI' 'CL-SEN' 'CL-SEM' 'CL-SEA' 'CL-CHP' 'CM' 'CN' 'CO' 'CV' 'CD' 'CR'
 'CU' 'CY' 'CZ' 'DE' 'DJ' 'DK-DK1' 'DK-DK2' 'DK-BHM' 'DM' 'DO' 'DZ' 'EC'
 'EE' 'EG' 'EH' 'ER' 'ES' 'ES-CN-LP' 'ES-CN-HI' 'ES-CN-IG' 'ES-CN-TE'
 'ES-CN-GC' 'ES-CN-FV' 'ES-CN-LZ' 'ES-IB-IZ' 'ES-IB-MA' 'ES-IB-ME' 'ET'
 'FI' 'FJ' 'FK' 'FM' 'FO-MI' 'FR' 'FR-COR' 'GA' 'GB' 'GB-NIR' 'GB-ORK'
 'GB-ZET' 'GE' 'GF' 'GH' 'GL' 'GM' 'GN' 'GP' 'GQ' 'GR' 'GS' 'GT' 'GU' 'GW'
 'GY' 'HK' 'HM' 'HN' 'HT' 'HU' 'HR' 'ID' 'IE' 'IL' 'IM' 'IN-AN' 'IN-NO'
 'IN-WE' 'IN-EA' 'IN-SO' 'IN-NE' 'IQ' 'KY' 'IR' 'IS' 'IT-CNO' 'IT-C

In [9]:

import json
import geopandas as gpd

# Load your JSON data
with open('config/traceroutes/pmeter_tr/chi_to_buff.json') as f:
    data = json.load(f)

# Extract points (skip non-location entries like 'time', 'node_id')
points = []
for key, value in data.items():
    if isinstance(value, dict) and 'lat' in value and 'lon' in value:
        points.append({
            'ip': key,
            'lon': value['lon'],
            'lat': value['lat']
        })

# Create GeoDataFrame
gdf_points = gpd.GeoDataFrame(
    points,
    geometry=gpd.points_from_xy([p['lon'] for p in points], [p['lat'] for p in points]),
    crs="EPSG:4326"  # Standard lon/lat CRS
)

In [11]:
# Perform spatial join
result = gpd.sjoin(
    gdf_points,
    world_em_df,
    how="left",      # Keep all points (even if no state matches)
    predicate="within"  # Find points inside state polygons
)
print(result)
# Clean up the output
ip_zoneInfo = result[['ip', 'lon', 'lat', 'geometry', 'zoneName']]  # Adjust 'name' to your state column
print(ip_zoneInfo)

Empty GeoDataFrame
Columns: [geometry, index_right, zoneName, countryKey, countryName]
Index: []


KeyError: "['ip', 'lon', 'lat'] not in index"

In [48]:
import pandas as pd

df_ci = pd.read_csv('data/historical_data/2024_ZONES_DF.csv', index_col=False)
df_ci['datetime'] = pd.to_datetime(df_ci['datetime'], utc=True)
print(df_ci.columns)
print(df_ci.iloc[-1])

Index(['datetime', 'Country', 'zone_name', 'zone_id', 'ci', 'ci_lifecycle',
       'Carbon-free energy percentage (CFE%)',
       'Renewable energy percentage (RE%)', 'Data source', 'Data estimated',
       'Data estimation method'],
      dtype='object')
datetime                                           2024-12-31 23:00:00+00:00
Country                                                                  USA
zone_name                                                     Virgin Islands
zone_id                                                                   VI
ci                                                                    554.62
ci_lifecycle                                                          676.78
Carbon-free energy percentage (CFE%)                                    3.55
Renewable energy percentage (RE%)                                       3.55
Data source                                      Electricity Maps Estimation
Data estimated                                     

/var/folders/rn/lb6d61xx4_j9fhhnzl_6wkk40000gn/T/ipykernel_1622/655864792.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ci = pd.read_csv('data/historical_data/2024_ZONES_DF.csv', index_col=False)


In [49]:
day = pd.to_datetime('2024-12-31 23:00:00', utc=True)
df_ci_filtered = df_ci[df_ci['datetime'] == day]

In [50]:
final_result = pd.merge(
    ip_zoneInfo,
    df_ci_filtered[['zone_id', 'ci', 'ci_lifecycle']],
    left_on='zoneName',
    right_on='zone_id',
    how='left'  # Keep all IPs even if no CI data exists
)
print(final_result)

                ip      lon      lat                  geometry     zoneName  \
0  142.254.216.137 -78.8637  42.8801  POINT (-78.8637 42.8801)   US-NY-NYIS   
1    24.58.216.229 -78.8784  42.8864  POINT (-78.8784 42.8864)   US-NY-NYIS   
2     24.58.46.160 -74.0060  40.7128   POINT (-74.006 40.7128)   US-NY-NYIS   
3      24.58.32.62 -77.6088  43.1566  POINT (-77.6088 43.1566)   US-NY-NYIS   
4       66.109.6.2 -87.6298  41.8781  POINT (-87.6298 41.8781)  US-MIDA-PJM   
5     66.109.10.83 -87.6298  41.8781  POINT (-87.6298 41.8781)  US-MIDA-PJM   
6      4.68.37.137 -87.6298  41.8781  POINT (-87.6298 41.8781)  US-MIDA-PJM   
7      4.69.141.50 -96.7970  32.7767   POINT (-96.797 32.7767)  US-TEX-ERCO   
8      4.71.198.54 -96.7970  32.7767   POINT (-96.797 32.7767)  US-TEX-ERCO   
9   192.124.226.22 -97.7005  30.3328  POINT (-97.7005 30.3328)  US-TEX-ERCO   

       zone_id      ci  ci_lifecycle  
0   US-NY-NYIS  213.29        278.80  
1   US-NY-NYIS  213.29        278.80  
2   US-NY-NYI

In [51]:
# Filter for all data on 2024-12-31 (UTC)
start_date = pd.to_datetime("2024-12-31 00:00:00", utc=True)
end_date = pd.to_datetime("2024-12-31 23:59:59", utc=True)
df_day = df_ci[(df_ci['datetime'] >= start_date) & (df_ci['datetime'] <= end_date)]
print(df_day[df_day['zone_id'] == 'US'])

                      datetime Country zone_name zone_id      ci  \
8760 2024-12-31 00:00:00+00:00     USA       USA      US  324.37   
8761 2024-12-31 01:00:00+00:00     USA       USA      US  320.88   
8762 2024-12-31 02:00:00+00:00     USA       USA      US  316.54   
8763 2024-12-31 03:00:00+00:00     USA       USA      US  308.78   
8764 2024-12-31 04:00:00+00:00     USA       USA      US  300.83   
8765 2024-12-31 05:00:00+00:00     USA       USA      US  298.45   
8766 2024-12-31 06:00:00+00:00     USA       USA      US  296.36   
8767 2024-12-31 07:00:00+00:00     USA       USA      US  291.52   
8768 2024-12-31 08:00:00+00:00     USA       USA      US  288.50   
8769 2024-12-31 09:00:00+00:00     USA       USA      US  288.58   
8770 2024-12-31 10:00:00+00:00     USA       USA      US  294.28   
8771 2024-12-31 11:00:00+00:00     USA       USA      US  299.96   
8772 2024-12-31 12:00:00+00:00     USA       USA      US  305.58   
8773 2024-12-31 13:00:00+00:00     USA       USA

In [53]:
# Merge with your IP/zone data (from spatial join)
final_data = pd.merge(
    ip_zoneInfo,
    df_day,
    left_on='zoneName',
    right_on='zone_id',
    how='left'
)

# Drop redundant column
final_data = final_data.drop(columns=['zone_id'])
print(final_data)
print(type(final_data))
# print(final_data.groupby('ip')['Carbon intensity gCO₂eq/kWh (direct)'].count())

                  ip      lon      lat                  geometry     zoneName  \
0    142.254.216.137 -78.8637  42.8801  POINT (-78.8637 42.8801)   US-NY-NYIS   
1    142.254.216.137 -78.8637  42.8801  POINT (-78.8637 42.8801)   US-NY-NYIS   
2    142.254.216.137 -78.8637  42.8801  POINT (-78.8637 42.8801)   US-NY-NYIS   
3    142.254.216.137 -78.8637  42.8801  POINT (-78.8637 42.8801)   US-NY-NYIS   
4    142.254.216.137 -78.8637  42.8801  POINT (-78.8637 42.8801)   US-NY-NYIS   
..               ...      ...      ...                       ...          ...   
235   192.124.226.22 -97.7005  30.3328  POINT (-97.7005 30.3328)  US-TEX-ERCO   
236   192.124.226.22 -97.7005  30.3328  POINT (-97.7005 30.3328)  US-TEX-ERCO   
237   192.124.226.22 -97.7005  30.3328  POINT (-97.7005 30.3328)  US-TEX-ERCO   
238   192.124.226.22 -97.7005  30.3328  POINT (-97.7005 30.3328)  US-TEX-ERCO   
239   192.124.226.22 -97.7005  30.3328  POINT (-97.7005 30.3328)  US-TEX-ERCO   

                     dateti